In [39]:
# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# os and file types
import os
import sys
import datetime as dt
import json
import csv

# images and display
import base64, io, IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display, Math, Latex

# import the master data
spd=pd.read_csv("resources/inprocess/master.csv")
# drop unneeded columns
spd = spd.drop(axis = 1,columns = ["Unnamed: 0","mattenstrasse-area-1"])

In [40]:
#spd.head()
spd.species.nunique()

622

In [44]:
spd.lot.unique()

array(['X', 'none', nan, 'schwanen', 'flaess', 'taubenloch', 'port'],
      dtype=object)

In [58]:
spd.columns

Index(['date', 'species', 'year', 'location', 'biogeo', 'canton', 'river',
       'lot', 'forest', 'project', 'city', 'rch', 'rmp', 'rju', 'fam', 'res',
       'pri', 'inv', 'ode', 'ns', 'ongen', 'ni', 'cdf', 'biel', 'locname'],
      dtype='object')

In [65]:
spd.head()

,date,species,year,location,biogeo,canton,river,lot,forest,project,...,res,pri,inv,ode,ns,ongen,ni,cdf,biel,locname
0,2020-09-02,plantago-lanceolata,1,c,mp,be,madretschkanal,X,X,X,...,X,X,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1
1,2020-09-02,centaurea-nigra,1,c,mp,be,madretschkanal,X,X,X,...,X,X,X,X,ns,X,X,X,X,alleestrasse-madretschkanal-1
2,2020-09-02,plantago-media,1,c,mp,be,madretschkanal,X,X,X,...,X,X,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1
3,2020-09-02,chenopodium-album,1,c,mp,be,madretschkanal,X,X,X,...,X,X,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1
4,2020-09-02,centaurea-jacea,1,c,mp,be,madretschkanal,X,X,X,...,2.0,3.0,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1


In [ ]:
# summary statistics on the key categories - to discuss !

In [72]:
### Comparing lots, describing lots, there are four pulled out of the data here

sch = spd.loc[spd['lot'] == "schwanen"]

por = spd.loc[spd['lot'] == "port"]

tau = spd.loc[spd['lot'] == "taubenloch"]

fla = spd.loc[spd["lot"] == "flaess"]


# Compare numbers in key categories (here, not indigenous, watch list, black list, near threatened in MP)
nisch = sch.loc[sch['ni'] =='ni']
wlsch = sch.loc[sch['inv'] =='WL']
blsch = sch.loc[sch['inv'] =='BL']
ntmpsch =  sch.loc[sch['rmp'] =='NT']
vumpsch =  sch.loc[sch['rmp'] =='VU']

print(F"there is {ntmpsch.species.nunique()} species on the NT list")
print(F"there is {wlsch.species.nunique()} species on the watch list")
print(F"there is {blsch.species.nunique()} species on the black list")
print(F"there is {vumpsch.species.nunique()} species on the vulnerable list")
print(F"there are {nisch.species.nunique()} non indigenous non invasive species")


there is 1 species on the NT list
there is 0 species on the watch list
there is 4 species on the black list
there is 0 species on the vulnerable list
there are 3 non indigenous non invasive species


In [ ]:
# Describe Madretschkanal and Suze, compare them, describe cygnes renat area

In [ ]:
# describe pro nature renaturalization areas, compare to lots, to other renats, to everything else "berm"

In [51]:
sch.species.nunique()

98

2